In [ ]:
"""
This training script can be run both on a single gpu in debug mode,
and also in a larger training run with distributed data parallel (ddp).

To run on a single GPU, example:
$ python train.py --batch_size=32 --compile=False

To run with DDP on 4 gpus on 1 node, example:
$ torchrun --standalone --nproc_per_node=4 train.py

To run with DDP on 4 gpus across 2 nodes, example:
- Run on the first (master) node with example IP 123.456.123.456:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=0 --master_addr=123.456.123.456 --master_port=1234 train.py
- Run on the worker node:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=1 --master_addr=123.456.123.456 --master_port=1234 train.py
(If your cluster does not have Infiniband interconnect prepend NCCL_IB_DISABLE=1)
"""

import os
import time
import math
import pickle
import json
from contextlib import nullcontext

import pandas as pd

import numpy as np
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

from model import GPTConfig, GPT

# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O
out_dir = 'out-stock'
eval_interval = 100
log_interval = 100
eval_iters = 2
eval_only = False # if True, script exits right after the first eval
always_save_checkpoint = False # if True, always save a checkpoint after each eval
init_from = 'scratch' # 'scratch' or 'resume' or 'gpt2*'
# wandb logging
wandb_log = False # disabled by default
wandb_project = 'stock'
wandb_run_name = 'mini-gpt' # 'run' + str(time.time())
# data
dataset = 'stock'
gradient_accumulation_steps = 1 # used to simulate larger batch sizes
batch_size = 24 # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 5
# model
n_layer = 8
n_head = 8
n_embd = 512 # 参数估计：params= layer*(12*emb**2), 数据量估计dataSize= 10 *params
# dropout = 0.998728434 # for pretraining 0 is good, for finetuning try 0.1+
dropout = 0
# dropout = 1-dataSize/10/params
bias = False # do we use bias inside LayerNorm and Linear layers?
meta_vocab_size = n_embd // 2

# adamw optimizer
learning_rate = 1e-3 # max learning rate
max_iters = 1e6 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.99
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 100 # how many steps to warm up for
lr_decay_iters = 2000 # should be ~= max_iters per Chinchilla
min_lr = 1e-4 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla

# system
device = 'cpu' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
# -----------------------------------------------------------------------------
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
config = {k: globals()[k] for k in config_keys} # will be useful for logging
# -----------------------------------------------------------------------------

master_process = True


if master_process:
    os.makedirs(out_dir, exist_ok=True)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)


In [ ]:

datadir = os.path.join('data', dataset)

# meta数据
meta = {}
with open(os.path.join(datadir, 'meta.pkl'), 'r') as f:
    meta = json.load(f)
    meta_vocab_size = meta['vocab_size']
    meta_vocab_size = 4096
def decode(id):
    return meta['itos'][str(id)]
def encode(s):
    return [meta['stoi'][c] for c in s]


df_data = pd.read_csv(os.path.join(datadir, 'train.csv')).iloc[1:,:meta_vocab_size+1]

length = len(df_data)
pd_train_data = df_data.iloc[:int(length*0.95)]
pd_train_data.reset_index(drop=True, inplace=True)

pd_val_data = df_data.iloc[int(length*0.95)-block_size:int(length*0.98)]
pd_val_data = df_data.iloc[int(length*0.95)-block_size:]
pd_val_data.reset_index(drop=True, inplace=True)

pd_test_data = df_data.iloc[int(length*0.98)-block_size:]
pd_test_data.reset_index(drop=True, inplace=True)


pd_train_data

In [ ]:
pd_val_data

In [ ]:
pd_test_data

In [ ]:
def trans_frame_to_id(dataframe):
    train_data = dataframe.iloc[:, 1:]
    # 对所有行，都取前10个最大的
    def top_n(row, n):
        # return row.nlargest(n).values
        return row.nlargest(n).index.tolist()

    n = batch_size
    data_top_10 = train_data.apply(top_n, axis=1, n=n)

    # 将结果转换为 [266, 10] 的形状
    data_transformed = pd.DataFrame(data_top_10.tolist(), index=train_data.index)

    def to_id(row):
        return encode(row)
    
    data_transformed = data_transformed.apply(to_id, axis=1)
    data_transformed = torch.stack([torch.tensor(row) for row in data_transformed])
    return data_transformed

train_data = trans_frame_to_id(pd_train_data)
val_data = trans_frame_to_id(pd_val_data)

print(f'train.shape is {train_data.shape}, val.shape is {val_data.shape}')
train_data

In [ ]:
# 写个python程序，从pandas数据中获取一批训练数据：
# 1. 读取pandas数据A，格式是：[trade_date, label1, label2, label3，。。。]，date的样例有：20230104，label*是数字，样例有1.0399、0.9943
# 2. X是生成指定格式的shape = [shape, ]

import random


def get_batch(split, index=None):
    data = train_data if split == 'train' else val_data
    pd_data = pd_train_data if split == 'train' else pd_val_data
    # data = val_data

    if index is None:
        indices = torch.randint(len(data)-1-block_size, (batch_size, ))
        index_block = torch.randint(data.shape[-1], (batch_size, block_size + 1, 1))
        # print(index_block)

        # (batch, block)
        x = torch.stack([data[i:i+block_size] for i in indices])
        x = x.gather(2, index_block[:,:block_size,:]).squeeze(-1)
    
        # (batch, block)
        y = torch.stack([data[i+1:i+1+block_size] for i in indices])
        y = y.gather(2, index_block[:,1:,:]).squeeze(-1)
    else:
        index_block = torch.randint(data.shape[-1], (block_size, 1))
        x = data[index: index+block_size]
        x = x.gather(-1, index_block).squeeze(-1)
        x = torch.stack([x for i in range(data.shape[-1])])
        
        y_predict = data[index+block_size].reshape(x.shape[0], -1)
        y = torch.cat((x[:,1:], y_predict), dim=1)

    # index_first = index
    # if index is None:
    #     index_first = indices[0].item()
    # for i in range(block_size):
    #     print(f'x is date={pd_data.iloc[index_first+i, 0]}, chg={pd_data.loc[index_first+i, decode(x[0][i].item())]:<6}, code={decode(x[0][i].item())}, code_id={x[0][i].item()}')
    # print('----')
    # for i in range(block_size):
    #      print(f'y is date={pd_data.iloc[index_first+i+1, 0]}, chg={pd_data.loc[index_first+i+1, decode(y[0][i].item())]:<6}, code={decode(y[0][i].item())}, code_id={y[0][i].item()}')
    
    return x, y

x, y = get_batch('train', 0)
print(f'x.shape is {x.shape}, y.shape is {y.shape}')



In [ ]:
# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9

# model init
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=meta_vocab_size, dropout=dropout) # start with model_args from command line
if init_from == 'scratch':
    # init a new model from scratch
    print("Initializing a new model from scratch")
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)


model.to(device)

# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

# optimizer
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)

checkpoint = None # free up memory

# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            data = train_data if split == 'train' else val_data
            X, Y = get_batch(split, random.randint(0, len(data)-block_size-1))
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

# logging
if wandb_log and master_process:
    import wandb
    wandb.init(project=wandb_project, name=wandb_run_name, config=config)

X, Y = get_batch('train', 1)


print(f'X is {X}')
print(f'Y is {Y}')
logits, loss = model(X, Y)
print(logits.shape)
# print(f'logits is {logits}, shape is {logits.shape}')
# print(f'loss is {loss}, loss.shape is {loss.shape}')



In [ ]:


# training loop
# X, Y = get_batch('train') # fetch the very first batch
t0 = time.time()
local_iter_num = 0 # number of iterations in the lifetime of this process
raw_model = model # unwrap DDP container if needed
running_mfu = -1.0

size_train_data = len(train_data) - block_size

while True:
    # evaluate the loss on train/val sets and write checkpoints
    if iter_num % eval_interval == 0 and master_process:
        losses = estimate_loss()
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        
        if losses['train'] < best_val_loss or always_save_checkpoint:
            best_val_loss = losses['train']
            if iter_num >= 0:
                checkpoint = {
                    'model': raw_model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'config': config,
                }
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))
                
    if iter_num == 0 and eval_only:
        break
    
    X, Y = get_batch('train', iter_num % size_train_data)
    logits, loss = model(X, Y)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    # timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0 and master_process:
        # get loss as float. note: this is a CPU-GPU sync point
        # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
        lossf = loss.item() * gradient_accumulation_steps
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms, epoch {int(iter_num/size_train_data)}, index {iter_num%size_train_data}")
    iter_num += 1
    local_iter_num += 1

    # termination conditions
    if iter_num > max_iters:
        break

